<a href="https://colab.research.google.com/github/IamWafula/kendickLamarVDrake/blob/main/sentiment_roberta_prediction_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the transformers library
!pip install datasets transformers==4.28.0
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 19.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [2]:
# Import required packages
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts["input_ids"])

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [3]:
# Load tokenizer and model, create trainer
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [4]:
# Create list of texts (can be imported from .csv, .xls etc.)
pred_texts = [
    "[Intro: Bēkon]",
    "Is it wickedness?",
    "Is it weakness?",
    "You decide",
    "Are we gonna live or die?",
    "",
    "[Verse: Kendrick Lamar]",
    "So I was takin' a walk the other day",
    "And I seen a woman—a blind woman",
    "Pacin' up and down the sidewalk",
    "She seemed to be a bit frustrated",
    "As if she had dropped somethin' and havin' a hard time findin' it",
    "So after watchin' her struggle for a while",
    "I decide to go over and lend a helping hand, you know?",
    "\"Hello, ma'am, can I be of any assistance?",
    "It seems to me that you have lost something",
    "I would like to help you find it\"",
    "She replied, \"Oh, yes, you have lost something",
    "You've lost your life\"",
    "*Gunshot*",
    "",
    "[Bridge: Bēkon]",
    "Is it wickedness?",
    "",
    "[Outro: Eric Bolling & Kimberly Guilfoyle]",
    "Uh, Lamar state his views—",
    "Uh, Lamar state his views—",
    "Uh, Lamar state his views—",
    "Uh, Lamar state his views—",
    "Uh, Lamar state his views on police brutality with that line in the song, quote",
    "\"And we hate the po-po, wanna kill us in the street fo' sho'\"",
    "Oh, please, ugh, I don't like it"
]

In [10]:
pred_texts = [
    "I got, I got, I got, I got",
    "Loyalty, got royalty inside my DNA",
    "Quarter piece, got war, and peace inside my DNA",
    "I got power, poison, pain, and joy inside my DNA",
    "I got hustle, though, ambition flow inside my DNA",
    "I was born like this",
    "Since one like this, immaculate conception",
    "I transform like this, perform like this",
    "Was Yeshua new weapon",
    "I don't contemplate, I meditate",
    "Then off your-, off your head",
    "This that put-the-kids-to-bed",
    "This that I got, I got, I got, I got",
    "Realness, I just kill sh- 'cause it's in my DNA",
    "I got millions, I got riches buildin' in my DNA",
    "I got dark, I got evil that rot inside my DNA",
    "I got off, I got troublesome heart inside my DNA",
    "I just win again, then, win again like Wimbledon I serve",
    "Yeah, that's him again, the sound that engine in is like a bird",
    "You see fireworks and Corvette tire skrrt the boulevard",
    "I know how you work, I know just who you are",
    "See, you's a, you's a, you's a",
    "B-, your hormones prolly switch inside your DNA",
    "Problem is, all that sucker shit inside your DNA",
    "Daddy prolly snitched, heritage inside your DNA",
    "Backbone don't exist, born outside a jellyfish, I gauge",
    "See, my pedigree most definitely don't tolerate the front",
    "Sh- I've been through prolly offend you",
    "This is Paula's oldest son",
    "I know murder, conviction, burners, boosters",
    "Burglars, ballers, dead, redemption",
    "Scholars, fathers dead with kids, and",
    "I wish I was fed forgiveness",
    "Yeah, yeah, yeah, yeah, soldier's DNA (I'ma soldier DNA)",
    "Born inside the beast",
    "My expertise checked out in second grade",
    "When I was nine, on cell, motel",
    "We didn't have nowhere to stay",
    "At 29, I've done so well, hit cartwheel in my estate",
    "And I'm gon' shine like I'm supposed to antisocial, extrovert",
    "And excellent mean the extra work",
    "And absentness what the fuck you heard",
    "And pessimists never struck my nerve",
    "And Nazareth gonna plead this case",
    "The reason my power's here on earth",
    "Salute the truth, when the prophet say",
    "I-I got loyalty, got royalty inside my DNA (this is why I say that hip hop)",
    "(Has done more damage to young African Americans) (I got loyalty, got royalty inside my DNA)",
    "I live a better life, I'm rollin' several dice, f- your life",
    "(Than racism in recent years) (I got loyalty, got royalty inside my DNA)",
    "(I live a be-, - your life)",
    "This is my heritage, all I'm inheritin' (five, four, three, two, one)",
    "Money and power, the Mecca of marriages",
    "Tell me somethin' (give me some ganja)",
    "You motherfuckers can't tell me nothin'",
    "I'd rather die than to listen to you",
    "My DNA not for imitation, your DNA, an abomination",
    "This how it is when you're in the Matrix",
    "Dodgin' bullets, reapin' what you sow",
    "And stackin' up the footage, livin' on the go, and sleepin' in a villa",
    "Sippin' from a Grammy and walkin' in the buildin'",
    "Diamond in the ceilin', marble on the floors",
    "Beach inside the window, peekin' out the window",
    "Baby in the pool, godfather goals",
    "Only Lord knows I've been goin' hammer",
    "Dodgin' paparazzi, freakin' through the cameras",
    "Eat at Four Daughters, Brock wearin' sandals",
    "Yoga on a Monday, stretchin' to Nirvana",
    "Watchin' all the snakes, curvin' all the fakes",
    "Phone never on, I don't conversate",
    "I don't compromise, I just penetrate",
    "Sex, money, murder, these are the breaks",
    "These are the times, level number nine",
    "Look up in the sky, .10 is on the way",
    "Sentence on the way, killings on the way",
    "Motherf-ker, I got winners on the way",
    "You ain't sh- without a body on your belt",
    "You ain't sh- without a ticket on your plate",
    "You ain't sick enough to pull it on yourself",
    "You ain't rich enough to hit the lot and skate",
    "Tell me when destruction gonna be my fate",
    "Gonna be your fate, gonna be our faith?",
    "Peace to the world, let it rotate",
    "Sex, money, murder, our DNA"
]

In [ ]:
# Example: Import data from csv-file stored on Google Drive

#from google.colab import drive
#drive.mount('/content/drive')


#file_name = "/content/drive/MyDrive/Colab Notebooks/your-filename.csv"
#text_column = "text"

#df_pred = pd.read_csv(file_name)
#pred_texts = df_pred[text_column].dropna().astype('str').tolist()

In [11]:
# Tokenize texts and create prediction data set
tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [12]:
# Run predictions
predictions = trainer.predict(pred_dataset)

In [7]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [13]:
# Create DataFrame with texts, predictions, labels, and scores
df = pd.DataFrame(list(zip(pred_texts,preds,labels,scores)), columns=['text','pred','label','score'])
df

,text,pred,label,score
0,"I got, I got, I got, I got",1,POSITIVE,0.989007
1,"Loyalty, got royalty inside my DNA",0,NEGATIVE,0.990554
2,"Quarter piece, got war, and peace inside my DNA",0,NEGATIVE,0.990458
3,"I got power, poison, pain, and joy inside my DNA",1,POSITIVE,0.688861
4,"I got hustle, though, ambition flow inside my DNA",0,NEGATIVE,0.984316
5,I was born like this,0,NEGATIVE,0.935417
6,"Since one like this, immaculate conception",1,POSITIVE,0.993733
7,"I transform like this, perform like this",1,POSITIVE,0.996233
8,Was Yeshua new weapon,1,POSITIVE,0.986487
9,"I don't contemplate, I meditate",1,POSITIVE,0.996848
